In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241212_01_MLlib_regression").getOrCreate()

24/12/13 10:22:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# data load

In [4]:
train_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_train.csv')

In [5]:
test_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_test.csv')

In [6]:
# 지도와 비지도 구분 - 라벨 우리가 알고싶은 값. 
#어떤 것을 선택하는지에 따라 모델 성능 차이. 가장 효과적인 . 성능을 선택해서 만든 모델이  우수 

In [7]:
#sale price: label
#features: ????


In [8]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

# 전처리 -  결측치 =0 으로 

In [ ]:
# 타입변환 ** 필요 

In [ ]:
#GarageArea, GarageCars -  integer로 cast 

In [10]:
train_df= train_df.withColumn("GarageArea", train_df["GarageArea"].cast("integer"))
test_df= test_df.withColumn("GarageArea", test_df["GarageArea"].cast("integer"))

In [11]:
train_df= train_df.withColumn("GarageCars", train_df["GarageCars"].cast("integer"))
test_df= test_df.withColumn("GarageCars", test_df["GarageCars"].cast("integer"))

In [12]:
#결측치 0 
train_df= train_df.fillna(0)
test_df = test_df.fillna(0)

# 인코딩 - 문자형 > 숫자형  1,2,3,4

In [13]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

In [14]:
string_columns = ['Neighborhood']# 원하는 컬럼 더 넣기 
# 1,2,3,4 로 값을 단순화(문자로 되어있음 )
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index")   for col in string_columns]

In [15]:
# onehotencoding : 범주형변수 1,2,3,4, -> 1로 바꾸는 인코딩
# 모두 1로 바꾼다.
#다른 컬럼 추가해보기 
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded")   for col in string_columns]

# features selection

In [16]:
numeric_columns=["LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

In [17]:
#문자형(인코딩)+ 숫자형 피처를 결합한 모델입력 생성 
assembler_inputs = [col+"_encoded" for col in string_columns] + numeric_columns

#   assembler

In [18]:
assembler= VectorAssembler(inputCols= assembler_inputs, outputCol="features")

# Label selection 

In [19]:
train_df= train_df.withColumnRenamed("SalePrice", "label")

# pipeline 설정 

### StringIndex + OnhotEncoder +Assembler >> 하나의 SparkML Pipeline 으로 결합 

In [20]:
from pyspark.ml import Pipeline 

In [21]:
pipeline= Pipeline(stages = indexers + encoders + [assembler] ) #깡통을 모아 집합 만들기  # 위 파이프라인에 이름만 지정해 놓음 

In [ ]:
#pipeline 실행 1

In [22]:
pipeline_model = pipeline.fit(train_df) #훈련 데이터 넣기  

In [23]:
train_transformed= pipeline_model.transform(train_df) #학습 시켜주기  # 파이프라인 모델  #train데이터가 모델에 들어가기 위해 적재됨. 

In [ ]:
# 여기까지는 전처리

# 모델 생성 ::  회귀 모델 학습 > 평가 > 예측 

In [24]:
from pyspark.ml.regression import LinearRegression

In [25]:
lr= LinearRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_transformed)
# 예측모뎅 

24/12/13 10:27:34 WARN Instrumentation: [c0907f1b] regParam is zero, which might cause numerical instability and overfitting.
24/12/13 10:27:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/13 10:27:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/13 10:27:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/13 10:27:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


numeric_columns 의 2개 컬럼이 string임. > 형변환 수행하기 

In [26]:
# 평가 데이터를 이용한 평가 > FIT 모델 맞춤 과정, 테스트 데이터에 의해 규칙이 변화해버림 

test_transformed= pipeline_model.transform(test_df) #여기는 fit 안함 . 적용만 하기 . **
predictions= lr_model.transform(test_transformed)
 # pipeline_model, lr_model 재활용 할 때 필요 

In [28]:
# 예측값 확인 
predictions.select("id" , "features", "prediction").show(10, truncate=False) #saleprice 로 실제 값이랑 비교해볼 수 있음. 

+----+-------------------------------------------------------------------------------------------------------+------------------+
|id  |features                                                                                               |prediction        |
+----+-------------------------------------------------------------------------------------------------------+------------------+
|1461|(34,[0,24,25,26,27,28,29,31,32,33],[1.0,11622.0,5.0,6.0,1961.0,1961.0,896.0,896.0,1.0,730.0])          |114113.60325331613|
|1462|(34,[0,24,25,26,27,28,29,31,32,33],[1.0,14267.0,6.0,6.0,1958.0,1958.0,1329.0,1329.0,1.0,312.0])        |156145.8445868329 |
|1463|(34,[5,24,25,26,27,28,29,30,31,32,33],[1.0,13830.0,5.0,5.0,1997.0,1998.0,928.0,701.0,1629.0,2.0,482.0])|168254.6697326172 |
|1464|(34,[5,24,25,26,27,28,29,30,31,32,33],[1.0,9978.0,6.0,6.0,1998.0,1998.0,926.0,678.0,1604.0,2.0,470.0]) |186898.44701529457|
|1465|(34,[18,24,25,26,27,28,29,31,32,33],[1.0,5005.0,8.0,5.0,1992.0,1992.0,1280.0,1280.0,

In [ ]:
#결과 저장 

In [35]:
predictions.select("id" , "prediction").withColumnRenamed('prediction', 'SalePrice').write.csv('data/output/house_prediction.csv', header=True, mode= 'overwrite')

In [ ]:
# 예측값 읽어서 분석 , 차트 

In [ ]:
#예측 모델의 활용
1. 파이프라인 저장 > 로컬,data/output/ > 모델 저장소에 저장 
2. 모델 저장 > 로컬> 모델저장소에 저장 

In [39]:
model_save_path= 'data/output/boston_housing_lr_model'
pipeline_save_path= 'data/output/boston_housing_pipeline_model'
pipeline_model.write().overwrite().save(pipeline_save_path)
lr_model.write().overwrite().save(model_save_path)

# 모델, 파이프라인 로드 

In [44]:
from pyspark.ml import PipelineModel
from pyspark.ml.regression import LinearRegressionModel

In [45]:
loaded_pipline= PipelineModel.load(pipeline_save_path)
loaded_pipline

PipelineModel_4b9a27b3d117

In [46]:
loaded_model= LinearRegressionModel.load(model_save_path)
loaded_model

LinearRegressionModel: uid=LinearRegression_b6c88896ea3d, numFeatures=34

# 새로운 데이터로 예측

1. 새로운 데이터 >>> ???
2. 파이프라인모델을 이용해서 변환
3. 모델에 넣어 예측

In [43]:
import pandas as pd
# 새로운 데이터 샘플 생성
data = {
    "Id": [1461],
    "MSSubClass": [20],
    "MSZoning": ["RH"],
    "LotFrontage": [80],
    "LotArea": [11622],
    "Street": ["Pave"],
    "Alley": [None],  # NA를 None으로 표현
    "LotShape": ["Reg"],
    "LandContour": ["Lvl"],
    "Utilities": ["AllPub"],
    "LotConfig": ["Inside"],
    "LandSlope": ["Gtl"],
    "Neighborhood": ["NAmes"],
    "Condition1": ["Feedr"],
    "Condition2": ["Norm"],
    "BldgType": ["1Fam"],
    "HouseStyle": ["1Story"],
    "OverallQual": [5],
    "OverallCond": [6],
    "YearBuilt": [1961],
    "YearRemodAdd": [1961],
    "RoofStyle": ["Gable"],
    "RoofMatl": ["CompShg"],
    "Exterior1st": ["VinylSd"],
    "Exterior2nd": ["VinylSd"],
    "MasVnrType": [None],  # None은 NA를 의미
    "MasVnrArea": [0],
    "ExterQual": ["TA"],
    "ExterCond": ["TA"],
    "Foundation": ["CBlock"],
    "BsmtQual": ["TA"],
    "BsmtCond": ["TA"],
    "BsmtExposure": ["No"],
    "BsmtFinType1": ["Rec"],
    "BsmtFinSF1": [468],
    "BsmtFinType2": ["LwQ"],
    "BsmtFinSF2": [144],
    "BsmtUnfSF": [270],
    "TotalBsmtSF": [882],
    "Heating": ["GasA"],
    "HeatingQC": ["TA"],
    "CentralAir": ["Y"],
    "Electrical": ["SBrkr"],
    "1stFlrSF": [896],
    "2ndFlrSF": [0],
    "LowQualFinSF": [0],
    "GrLivArea": [896],
    "BsmtFullBath": [0],
    "BsmtHalfBath": [0],
    "FullBath": [1],
    "HalfBath": [0],
    "BedroomAbvGr": [2],
    "KitchenAbvGr": [1],
    "KitchenQual": ["TA"],
    "TotRmsAbvGrd": [5],
    "Functional": ["Typ"],
    "Fireplaces": [0],
    "FireplaceQu": [None],  # NA를 None으로 표현
    "GarageType": ["Attchd"],
    "GarageYrBlt": [1961],
    "GarageFinish": ["Unf"],
    "GarageCars": [1],
    "GarageArea": [730],
    "GarageQual": ["TA"],
    "GarageCond": ["TA"],
    "PavedDrive": ["Y"],
    "WoodDeckSF": [140],
    "OpenPorchSF": [0],
    "EnclosedPorch": [0],
    "3SsnPorch": [0],
    "ScreenPorch": [120],
    "PoolArea": [0],
    "PoolQC": [None],  # NA를 None으로 표현
    "Fence": ["MnPrv"],
    "MiscFeature": [None],  # NA를 None으로 표현
    "MiscVal": [0],
    "MoSold": [6],
    "YrSold": [2010],
    "SaleType": ["WD"],
    "SaleCondition":["Normal"]
}
pd.DataFrame(data).to_csv('data/new_test_data.csv', index=False)   
    

In [47]:
new_test_data = spark.read.csv("data/new_test_data.csv", header=True, inferSchema=True)

In [48]:
#필요한 특성만 선택(파이프라인에서 사용된 특성들)
#수치형 컬럼 + 범주형 컬럼 정의 
selected_features = [
    "LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea", "Neighborhood"
]

In [49]:
# 데이터 타입 변환 및 필요한 특성 선택
new_test_data = new_test_data.withColumn("GarageCars", new_test_data["GarageCars"].cast("integer"))
new_test_data = new_test_data.withColumn("GarageArea", new_test_data["GarageArea"].cast("integer"))

In [51]:
#파이프라인에 새로운 데이터를 넣어 준다.
new_pipe_data =loaded_pipline.transform(new_test_data)

In [ ]:
# 파이프라인이 변환한 데이터를 모델에 넣어 준다.

In [56]:
new_pred = loaded_model.transform( new_pipe_data ) #1건

In [ ]:
# 예측 수행 

In [57]:
new_pred.select("prediction").show()

+------------------+
|        prediction|
+------------------+
|114113.60325331613|
+------------------+



In [ ]:
# csv저장 

In [ ]:
#Log 를 남긴다 - csv 저장, 데이터베이스에 저장 , 데이터베이스에 저장 "외부저장", 하둡 분산파일시스템  hdfs, kafka(streaming)